In [1]:
import os
import sys
import ast
import json
from collections import defaultdict

import cv2
import numpy as np
import pandas as pd
import neuroglancer
from tqdm import tqdm
from skimage import io
import imagesize

sys.path.append(os.path.join(os.getcwd(), '../'))
from utilities.file_location import FileLocationManager
from utilities.sqlcontroller import SqlController
from utils import get_structure_number

Connecting dklab@192.168.1.12:3306


In [33]:
animal = 'MD589'
downsample_factor = 16
all_structures = False

# OUTPUT_DIR_PATH = os.path.join(os.path.expanduser('~'))
OUTPUT_DIR_PATH = os.path.join('./')
CSV_DIR_PATH = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data'
IMAGE_DIR_PATH = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH1/full'
    
sqlController = SqlController(animal)
resolution = sqlController.scan_run.resolution
aligned_shape = np.array((sqlController.scan_run.width, sqlController.scan_run.height))
num_section = len(os.listdir(IMAGE_DIR_PATH))

downsampled_aligned_shape = np.round(aligned_shape / downsample_factor).astype(int)
scales = np.array([resolution * downsample_factor, resolution * downsample_factor, 20]) * 1000

## Get the annotation points

In [31]:
csvfile = os.path.join(CSV_DIR_PATH, f'{animal}.formatted.structures.csv')
df = pd.read_csv(csvfile)
#csvoutfile = os.path.join(CSV_DIR_PATH, f'{animal}.formatted.structures.csv')
#df.to_csv(csvoutfile, index=False)
structures = df['structure'].unique()
sections = df['section'].unique()
section_structure_vertices = defaultdict(dict)
for index, row in df.iterrows():
    SN = row['section']
    ST = row['structure']
    section_structure_vertices[SN][ST] = row['vertices']

In [35]:
c = 94
for section in sections:
    print(c, str(section).zfill(3) + '.tif')
    c += 1

94 094.tif
95 095.tif
96 096.tif
97 097.tif
98 098.tif
99 099.tif
100 100.tif
101 101.tif
102 102.tif
103 103.tif
104 104.tif
105 105.tif
106 106.tif
107 107.tif
108 108.tif
109 109.tif
110 110.tif
111 111.tif
112 112.tif
113 113.tif
114 114.tif
115 115.tif
116 116.tif
117 117.tif
118 118.tif
119 119.tif
120 120.tif
121 121.tif
122 122.tif
123 123.tif
124 124.tif
125 125.tif
126 126.tif
127 127.tif
128 128.tif
129 129.tif
130 130.tif
131 131.tif
132 132.tif
133 133.tif
134 134.tif
135 135.tif
136 136.tif
137 137.tif
138 138.tif
139 139.tif
140 140.tif
141 141.tif
142 142.tif
143 143.tif
144 144.tif
145 145.tif
146 146.tif
147 147.tif
148 148.tif
149 149.tif


## Reproduce create_clean transform

In [34]:
section_offset = {}
for file_name in tqdm(sorted(os.listdir(IMAGE_DIR_PATH))):
    filepath = os.path.join(IMAGE_DIR_PATH, file_name)
    
    # Use imread is too slow for full res images
    width, height = imagesize.get(filepath)
    downsampled_shape = np.round(np.array((width, height)) / downsample_factor)
    
    section = int(file_name.split('.')[0])
    section_offset[section] = (downsampled_aligned_shape - downsampled_shape) // 2

  0%|          | 0/5 [00:00<?, ?it/s]


error: unpack requires a buffer of 2 bytes

## Reproduce create_alignment transform

In [ ]:
def parse_elastix_parameter_file(filepath):
    """
    Parse elastix parameter result file.
    """
    def parameter_elastix_parameter_file_to_dict(filename):
        d = {}
        with open(filename, 'r') as f:
            for line in f.readlines():
                if line.startswith('('):
                    tokens = line[1:-2].split(' ')
                    key = tokens[0]
                    if len(tokens) > 2:
                        value = []
                        for v in tokens[1:]:
                            try:
                                value.append(float(v))
                            except ValueError:
                                value.append(v)
                    else:
                        v = tokens[1]
                        try:
                            value = (float(v))
                        except ValueError:
                            value = v
                    d[key] = value
            return d

    d = parameter_elastix_parameter_file_to_dict(filepath)

    # For alignment composition script
    rot_rad, x_mm, y_mm = d['TransformParameters']
    center = np.array(d['CenterOfRotationPoint']) / np.array(d['Spacing'])
    # center[1] = d['Size'][1] - center[1]

    xshift = x_mm / d['Spacing'][0]
    yshift = y_mm / d['Spacing'][1]

    R = np.array([[np.cos(rot_rad), -np.sin(rot_rad)],
                  [np.sin(rot_rad), np.cos(rot_rad)]])
    shift = center + (xshift, yshift) - np.dot(R, center)
    T = np.vstack([np.column_stack([R, shift]), [0, 0, 1]])
    return T


def load_consecutive_section_transform(stack, moving_fn, fixed_fn):
    """
    Load pairwise transform.

    Returns:
        (3,3)-array.
    """
    assert stack is not None
    fileLocationManager = FileLocationManager(stack)
    elastix_output_dir = fileLocationManager.elastix_dir
    param_fp = os.path.join(elastix_output_dir, moving_fn + '_to_' + fixed_fn, 'TransformParameters.0.txt')
    #sys.stderr.write('Load elastix-computed transform: %s\n' % param_fp)
    if not os.path.exists(param_fp):
        raise Exception('Transform file does not exist: %s to %s, %s' % (moving_fn, fixed_fn, param_fp))
    transformation_to_previous_sec = parse_elastix_parameter_file(param_fp)

    return transformation_to_previous_sec

def parse_elastix(animal):
    """
    After the elastix job is done, this goes into each subdirectory and parses the Transformation.0.txt file
    Args:
        animal: the animal
    Returns: a dictionary of key=filename, value = coordinates
    """
    fileLocationManager = FileLocationManager(animal)
    DIR = fileLocationManager.prep
    INPUT = os.path.join(DIR, 'CH1', 'thumbnail_cleaned')

    image_name_list = sorted(os.listdir(INPUT))
    anchor_idx = len(image_name_list) // 2
    # anchor_idx = len(image_name_list) - 1
    transformation_to_previous_sec = {}

    for i in range(1, len(image_name_list)):
        fixed_fn = os.path.splitext(image_name_list[i - 1])[0]
        moving_fn = os.path.splitext(image_name_list[i])[0]
        transformation_to_previous_sec[i] = load_consecutive_section_transform(animal, moving_fn, fixed_fn)

    transformation_to_anchor_sec = {}
    # Converts every transformation
    for moving_idx in range(len(image_name_list)):
        if moving_idx == anchor_idx:
            transformation_to_anchor_sec[image_name_list[moving_idx]] = np.eye(3)
        elif moving_idx < anchor_idx:
            T_composed = np.eye(3)
            for i in range(anchor_idx, moving_idx, -1):
                T_composed = np.dot(np.linalg.inv(transformation_to_previous_sec[i]), T_composed)
            transformation_to_anchor_sec[image_name_list[moving_idx]] = T_composed
        else:
            T_composed = np.eye(3)
            for i in range(anchor_idx + 1, moving_idx + 1):
                T_composed = np.dot(transformation_to_previous_sec[i], T_composed)
            transformation_to_anchor_sec[image_name_list[moving_idx]] = T_composed

    return transformation_to_anchor_sec

def create_warp_transforms(transforms, downsample_factor=32):
    def convert_2d_transform_forms(arr):
        return np.vstack([arr, [0, 0, 1]])
    
    transforms_scale_factor = 32 / downsample_factor 
    tf_mat_mult_factor = np.array([[1, 1, transforms_scale_factor], [1, 1, transforms_scale_factor]])
    transforms_to_anchor = {}
    for img_name, tf in transforms.items():
        transforms_to_anchor[img_name] = convert_2d_transform_forms(np.reshape(tf, (3, 3))[:2] * tf_mat_mult_factor) 

    return transforms_to_anchor

transforms = parse_elastix(animal)
warp_transforms = create_warp_transforms(transforms, downsample_factor)
ordered_transforms = sorted(warp_transforms.items())

section_transform = {}
for section, transform in ordered_transforms:
    section_num = int(section.split('.')[0])
    transform = np.linalg.inv(transform)
    section_transform[section_num] = transform

## Alignment of annotation coordinates

In [ ]:
'''
(x', y') = (x * sx + y * ry + tx, x * rx + y * sy + ty)
'sx': T[0, 0], 'sy': T[1, 1], 'rx': T[1, 0], 'ry': T[0, 1], 'tx': T[0, 2], 'ty': T[1, 2]
'''
def transform_create_alignment(points, transform):
    a = np.hstack((points, np.ones((points.shape[0], 1))))
    b = transform.T[:, 0:2]
    c = np.matmul(a, b)
    return c

aligned_section_structure_polygons = defaultdict(dict)
for section in section_structure_vertices:
    for structure in section_structure_vertices[section]:
        points = np.array(section_structure_vertices[section][structure]) // downsample_factor
        points = points + section_offset[section] # create_clean offset
        points = transform_create_alignment(points, section_transform[section]) # create_alignment transform
        aligned_section_structure_polygons[section][structure] = [points]

In [ ]:
import pickle

vertices_path = os.path.join(OUTPUT_DIR_PATH, f'{animal}_aligned_section_structure_polygons_down{downsample_factor}.pickle')
with open(vertices_path, 'wb') as file:
    pickle.dump(aligned_section_structure_polygons, file)

To this point, aligned_section_structure_polygons variable contains the aligned polygon vertices for each structure in each section. 
From now on, we introduce how to draw these points to numpy array or neuroglancer

## Draw in a numpy volume

In [ ]:
def draw_numpy(section_structure_polygons, section_start, section_end):
    volume = np.zeros((downsampled_aligned_shape[1], downsampled_aligned_shape[0], section_end - section_start), dtype=np.uint8)
    for section in tqdm(range(section_start, section_end)):
        if section in section_structure_polygons:
            template = np.zeros((downsampled_aligned_shape[1], downsampled_aligned_shape[0]), dtype=np.uint8)
            for structure in section_structure_polygons[section]:
                polygons = section_structure_polygons[section][structure]
                for polygon in polygons:
                    color = get_structure_number(structure)
#                     cv2.polylines(template, [polygon.astype(np.int32)], True, color, 1)
                    for point in polygon:
                        cv2.circle(template, tuple(point.astype(np.int32)), 0, color, -1)

            volume[:, :, section - section_start - 1] = template
        
    volume = np.swapaxes(volume, 0, 1)
    return volume

volume = draw_numpy(aligned_section_structure_polygons, 0, num_section)

In [ ]:
numpy_path = os.path.join(OUTPUT_DIR_PATH, f'{animal}_annotations_down{downsample_factor}.npy')
with open(numpy_path, 'wb') as file:
    np.save(file, volume)

In [ ]:
all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=volume, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=scales), 
        voxel_offset=[0,0,0]
    ),
)

with viewer.txn() as s:
    s.layers['all'] = all_volume_layer

In [ ]:
with open('MD589_CVAT_down32.pkl', 'rb') as file:
    cvat_section_structure_polygons = pickle.load(file)
volume = draw_numpy(cvat_section_structure_polygons, 0, num_section)

all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=volume, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=scales), 
        voxel_offset=[0,0,0]
    ),
)

with viewer.txn() as s:
    s.layers['cvat'] = all_volume_layer